In [1]:
import geopandas as gpd
import pandas as pd

In [11]:
meteo = "https://www.data.gouv.fr/fr/datasets/r/edc8f427-4326-4e20-9fc1-91007a5a5a64"
df = pd.read_csv(meteo, sep=';')
#Ca c'est pour al émtéo que en ile de france, mais le soucis c'est que ce sont des prévisions, donc pas ok

In [ ]:
#Prévision que dans les 36 heures après avoir télécharger la base de donnée

In [4]:
df

,forecast,position,timestamp,2_metre_temperature,minimum_temperature_at_2_metres,maximum_temperature_at_2_metres,relative_humidity,total_water_precipitation,wind_speed,downward_short_wave_radiation_flux,surface_latent_heat_flux,surface_sensible_heat_flux,surface_net_solar_radiation,surface_net_thermal_radiation
0,2020-12-07T14:00:00+01:00,"49.15,2.175",2020-12-07T04:00:00+01:00,4.006860,3.680688,4.021570,91.158508,0.106445,1.729388,1246477.375,-312418.0,-263838.0,1057191.875,-181861.0
1,2020-12-08T00:00:00+01:00,"49.15,2.175",2020-12-07T04:00:00+01:00,1.707117,1.703333,2.054346,95.543274,0.169922,4.300119,1657436.750,-402448.0,-3537.0,1405896.000,-1020983.0
2,2020-12-08T07:00:00+01:00,"49.15,2.175",2020-12-07T04:00:00+01:00,1.112756,1.018610,1.110803,94.822922,0.201172,2.880767,1657436.750,-494576.0,137952.0,1405896.000,-1295895.0
3,2020-12-08T09:00:00+01:00,"49.15,2.175",2020-12-07T04:00:00+01:00,0.913110,0.916833,1.072290,95.780914,0.222656,2.786261,1663659.750,-522688.0,180464.0,1411226.250,-1364068.0
4,2020-12-08T13:00:00+01:00,"49.15,2.175",2020-12-07T04:00:00+01:00,2.716028,2.175562,2.721887,89.091888,0.253906,3.111121,3023134.000,-936064.0,-32128.0,2566557.500,-1575823.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43795,2020-12-08T14:00:00+01:00,"48.425,2.475",2020-12-07T04:00:00+01:00,3.255060,3.014978,3.263849,85.410934,0.468750,3.818275,5215643.000,-1315344.0,-280336.0,4385793.000,-3022128.0
43796,2020-12-08T18:00:00+01:00,"48.425,2.475",2020-12-07T04:00:00+01:00,2.708887,2.708887,3.081873,87.272156,0.558594,3.493914,5636240.000,-1525344.0,-381328.0,4739768.500,-3185328.0
43797,2020-12-08T19:00:00+01:00,"48.425,2.475",2020-12-07T04:00:00+01:00,2.176019,2.178949,2.727686,85.530945,0.562500,3.347610,5636240.000,-1557024.0,-357264.0,4739768.500,-3242784.0
43798,2020-12-07T08:00:00+01:00,"48.425,2.5",2020-12-07T04:00:00+01:00,2.004724,1.465662,2.004297,99.999344,0.053711,1.282431,0.000,-679.0,-32119.0,0.000,-4820.0


Les colonnes qui m'intéressent : 
date
etat_sol
ht_neige
Visibilité horizontale

In [26]:
meteo_gpd = 'https://www.data.gouv.fr/fr/datasets/r/8e834f69-0b43-439a-87bd-87ca80c84cd7'
stations = gpd.read_file(meteo_gpd)

In [27]:
stations

,surface_net_thermal_radiation,downward_short_wave_radiation_flux,2_metre_temperature,minimum_temperature_at_2_metres,surface_net_solar_radiation,timestamp,surface_latent_heat_flux,wind_speed,total_water_precipitation,relative_humidity,surface_sensible_heat_flux,forecast,maximum_temperature_at_2_metres,geometry
0,-181861.0,1246477.375,4.006860,3.680688,1057191.875,2020-12-07T04:00:00,-312418.0,1.729388,0.106445,91.158508,-263838.0,2020-12-07T14:00:00,4.021570,POINT (2.17500 49.15000)
1,-1020983.0,1657436.750,1.707117,1.703333,1405896.000,2020-12-07T04:00:00,-402448.0,4.300119,0.169922,95.543274,-3537.0,2020-12-08T00:00:00,2.054346,POINT (2.17500 49.15000)
2,-1295895.0,1657436.750,1.112756,1.018610,1405896.000,2020-12-07T04:00:00,-494576.0,2.880767,0.201172,94.822922,137952.0,2020-12-08T07:00:00,1.110803,POINT (2.17500 49.15000)
3,-1364068.0,1663659.750,0.913110,0.916833,1411226.250,2020-12-07T04:00:00,-522688.0,2.786261,0.222656,95.780914,180464.0,2020-12-08T09:00:00,1.072290,POINT (2.17500 49.15000)
4,-1575823.0,3023134.000,2.716028,2.175562,2566557.500,2020-12-07T04:00:00,-936064.0,3.111121,0.253906,89.091888,-32128.0,2020-12-08T13:00:00,2.721887,POINT (2.17500 49.15000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43795,-3022128.0,5215643.000,3.255060,3.014978,4385793.000,2020-12-07T04:00:00,-1315344.0,3.818275,0.468750,85.410934,-280336.0,2020-12-08T14:00:00,3.263849,POINT (2.47500 48.42500)
43796,-3185328.0,5636240.000,2.708887,2.708887,4739768.500,2020-12-07T04:00:00,-1525344.0,3.493914,0.558594,87.272156,-381328.0,2020-12-08T18:00:00,3.081873,POINT (2.47500 48.42500)
43797,-3242784.0,5636240.000,2.176019,2.178949,4739768.500,2020-12-07T04:00:00,-1557024.0,3.347610,0.562500,85.530945,-357264.0,2020-12-08T19:00:00,2.727686,POINT (2.47500 48.42500)
43798,-4820.0,0.000,2.004724,1.465662,0.000,2020-12-07T04:00:00,-679.0,1.282431,0.053711,99.999344,-32119.0,2020-12-07T08:00:00,2.004297,POINT (2.50000 48.42500)


In [28]:
meteo_idf = stations[['2_metre_temperature', 'timestamp', 'total_water_precipitation', 'forecast', 'geometry']]

In [32]:
meteo_idf

,2_metre_temperature,timestamp,total_water_precipitation,forecast,geometry
0,4.006860,2020-12-07T04:00:00,0.106445,2020-12-07T14:00:00,POINT (2.17500 49.15000)
1,1.707117,2020-12-07T04:00:00,0.169922,2020-12-08T00:00:00,POINT (2.17500 49.15000)
2,1.112756,2020-12-07T04:00:00,0.201172,2020-12-08T07:00:00,POINT (2.17500 49.15000)
3,0.913110,2020-12-07T04:00:00,0.222656,2020-12-08T09:00:00,POINT (2.17500 49.15000)
4,2.716028,2020-12-07T04:00:00,0.253906,2020-12-08T13:00:00,POINT (2.17500 49.15000)
...,...,...,...,...,...
43795,3.255060,2020-12-07T04:00:00,0.468750,2020-12-08T14:00:00,POINT (2.47500 48.42500)
43796,2.708887,2020-12-07T04:00:00,0.558594,2020-12-08T18:00:00,POINT (2.47500 48.42500)
43797,2.176019,2020-12-07T04:00:00,0.562500,2020-12-08T19:00:00,POINT (2.47500 48.42500)
43798,2.004724,2020-12-07T04:00:00,0.053711,2020-12-07T08:00:00,POINT (2.50000 48.42500)


In [31]:
meteo_idf.groupby('geometry').mean()

TypeError: '<' not supported between instances of 'Point' and 'Point'

In [21]:
df.dtypes 

forecast                               object
position                               object
timestamp                              object
2_metre_temperature                   float64
minimum_temperature_at_2_metres       float64
maximum_temperature_at_2_metres       float64
relative_humidity                     float64
total_water_precipitation             float64
wind_speed                            float64
downward_short_wave_radiation_flux    float64
surface_latent_heat_flux              float64
surface_sensible_heat_flux            float64
surface_net_solar_radiation           float64
surface_net_thermal_radiation         float64
dtype: object

In [5]:
accident_velo = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/3d5f2317-5afd-4a9f-a9c5-bd4fe0113f39')
accident_velo_idf = accident_velo[accident_velo['departement'].isin(['75','77','78','91','92','93','94','95'])]
accident_velo_idf
# Il y a un problème avec les latitudes et les longitudes nulles et les quelques erreurs

C:\Users\Vinciane\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (5,6,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,identifiant accident,date,mois,jour,heure,departement,commune,lat,lon,en agglomeration,...,existence securite,usage securite,obstacle fixe heurte,obstacle mobile heurte,localisation choc,manoeuvre avant accident,identifiant vehicule,type autres vehicules,manoeuvre autres vehicules,nombre autres vehicules
58,200500001606,2005-01-13,01 - janvier,3 - jeudi,11.0,78,78230,0.00000,0.00000,oui,...,NaN,NaN,NaN,Véhicule,Avant,Sans changement de direction,200500001606B02,VL seul,Sans changement de direction,1.0
61,200500001930,2005-01-15,01 - janvier,5 - samedi,15.0,95,95042,49.06400,2.32800,non,...,NaN,NaN,NaN,NaN,Arrière,Sans changement de direction,200500001930B02,Cyclomoteur <50cm3,"Même sens, même file",1.0
221,200500004591,2005-01-04,01 - janvier,1 - mardi,17.0,75,75001,0.00000,0.00000,oui,...,Casque,Oui,NaN,Véhicule,Avant,Sans changement de direction,200500004591B01,VL seul,Ouverture de porte,1.0
222,200500004597,2005-01-03,01 - janvier,0 - lundi,9.0,75,75005,0.00000,0.00000,oui,...,NaN,NaN,NaN,Véhicule,Arrière,Tournant à gauche,200500004597A01,Motocyclette,Sans changement de direction,1.0
223,200500004600,2005-01-03,01 - janvier,0 - lundi,17.0,75,75019,0.00000,0.00000,oui,...,NaN,NaN,NaN,Piéton,Avant,Sans changement de direction,200500004600A01,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65531,201800050291,2018-07-10,07 - juillet,1 - mardi,9.0,75,75101,48.86400,2.33134,oui,...,Casque,Non,Autre obstacle fixe sur chaussée,Véhicule,Avant droit,Sans changement de direction,201800050291B01,VL seul,Ouverture de porte,1.0
65532,201800050337,2018-11-17,11 - novembre,5 - samedi,12.0,75,75101,48.85854,2.34825,oui,...,Casque,Non déterminable,NaN,Véhicule,Côté gauche,Sans changement de direction,201800050337A01,Motocyclette > 125 cm3,Sans changement de direction,1.0
65533,201800050355,2018-02-01,02 - février,3 - jeudi,15.0,75,75102,48.86870,2.34133,oui,...,Casque,Oui,NaN,Véhicule,Avant,"Dans le couloir bus, dans le même sens",201800050355B01,VL seul,Tournant a droite,1.0
65534,201800050359,2018-02-12,02 - février,0 - lundi,9.0,75,75101,48.86470,2.34275,oui,...,Casque,Oui,NaN,Véhicule,Avant gauche,Traversant la chaussée,201800050359B01,Motocyclette > 50 cm3 et <= 125 cm3,Dépassant à gauche,1.0


In [6]:
list(accident_velo_idf)

['identifiant accident',
 'date',
 'mois',
 'jour',
 'heure',
 'departement',
 'commune',
 'lat',
 'lon',
 'en agglomeration',
 'type intersection',
 'type collision',
 'luminosite',
 'conditions atmosperiques',
 'type route',
 'circulation',
 'nb voies',
 'profil long route',
 'trace plan route',
 'largeur TPC',
 'largeur route',
 'etat surface',
 'amenagement',
 'situation',
 'categorie usager',
 'gravite accident',
 'sexe',
 'age',
 'motif deplacement',
 'existence securite',
 'usage securite',
 'obstacle fixe heurte',
 'obstacle mobile heurte',
 'localisation choc',
 'manoeuvre avant accident',
 'identifiant vehicule',
 'type autres vehicules',
 'manoeuvre autres vehicules',
 'nombre autres vehicules']

In [8]:
accident_velo_idf.groupby('conditions atmosperiques')["identifiant accident"].count()

conditions atmosperiques
Autre                     76
Brouillard - fumée        19
Neige - grêle             38
Normale                14917
Pluie forte              192
Pluie légère            1509
Temps couvert            627
Temps éblouissant        144
Vent fort - tempête       19
Name: identifiant accident, dtype: int64

In [9]:
accident_velo_idf.groupby('etat surface')["identifiant accident"].count()

etat surface
autre                    73
boue                      3
corps gras - huile       25
enneigée                 18
flaques                  14
inondée                   2
mouillée               2201
normale               14597
verglacée                33
Name: identifiant accident, dtype: int64